# Assignment2

## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|. | | |
|  | |. |

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


# Part 1: obtaining information from the web

### RSS parsing

Make a notebook that performs the following steps.

1. Create a script that retrieves all urls of rss feeds from <http://www.volkskrant.nl/rss/feeds/>. Use urllib2 and beautifulsoup for this. Store the urls in a list.
    * **update 2016**
    * As all Dutch sites, Volkskrant asks whether you accept cookies. This makes simple collecting webpages a lot harder. 
    * The code in the code cell below does the trick. 
    * After running this, I could collect further files from Volkskrant without additional cookie hassle.
2. Download all rss feeds and store them on disk.
3. Parse all RSS feeds using `lxml`. Create a list of  dicts with fields "channel", "url", "title", "date" in which you store this information for each item.
4. Compute some statistics about this dict: how many items, how many per channel, are there doubles (items occuring in several channels), etc.
5. Write this list as a csv file, store on disk, and upload to Google fusion tables.
6. Download all articles (once), parse out the text and store as pairs (date,text) in a list.
7. Count per day the number of words, and the number of unique words. Show this in a plot.

In [2]:
import cookielib # Thanks to http://stackoverflow.com/questions/29395407/enabling-cookies-with-urllib
import urllib2
import urllib
import requests
import os
from bs4 import BeautifulSoup
import re 


url = 'http://www.volkskrant.nl/rss/feeds/'

# with urllib2 and handling cookies
cookiejar= cookielib.LWPCookieJar()
opener= urllib2.build_opener( urllib2.HTTPCookieProcessor(cookiejar) )
response=opener.open(url)
html_doc= ' '.join(response.readlines())
 
rsssoup = BeautifulSoup(html_doc)

# test 
list_items=rsssoup.findAll('li')
len(list_items), list_items[:3]

(157, [<li class="alert__message fjs-alert__message">
  <a class="alert__close fjs-alert__close">
  <i class="icon icon--medium icon-close"></i>
  </a>
  <p class="alert__text fjs-message">
   					Oh snap! Change a few things up and try submitting again
   				</p>
  </li>, <li class="info-nav__item">
  <a class="info-nav__link" href="http://www.volkskrant.nl/het-weer/" itemprop="url" target="_self" title="Weer">
  <i class="icon icon-sun" role="presentation"></i>
  <span class="info-nav__weer" itemprop="name">Weer</span>
  </a>
  </li>, <li class="info-nav__item">
  <a class="info-nav__link" href=" http://www.z24.nl/koers-aex?partner=vk" itemprop="url" target="_blank" title="Beurs">
  <i class="icon icon-line-graph" role="presentation"></i>
  <span class="info-nav__beurs" itemprop="name">Beurs</span>
  </a>
  </li>])

### JSON parsing

1. Download <http://maartenmarx.nl/teaching/ISatWork/NoteBooks/consuming-json-data-from-a-web-service.ipynb>, remove all code blocks, and turn it into a notebook again. 
2. Check that what you did is correct and you did not remove too much using a notebook viewer.
3. Now extract all code from the downloaded notebook, save it to a file, and execute it as a Python script. Does it give errors? Is it syntactically correct?

### PDF parsing

1. Save a wordfile as PDF, open it in Python, extract all text. Describe the differences, if any. Try the same with a two column PDF file from the web. This exercise gets more interesting if you use _difficult_ PDF. Why not try <http://wch.github.io/latexsheet/latexsheet.pdf>?

* Is the word order still as it should be?
* What about the strange characters?

# Part 2 Python recap

Download [PythonRecap2.0.ipynb](PythonRecap2.0.ipynb),  and answer all questions as asked.